In [2]:
import numpy as np

from tensorflow import keras
from keras import layers
import os.path

from numpy import shape
from dataclasses import dataclass
from enum import Enum
from typing import List, Optional
import codecs
import os
import re
import librosa.feature
import matplotlib.pyplot as plt



In [3]:
def contains_any_index(root, a_list):
    for i, c in enumerate(a_list):
        if c.startswith(root):
            return i + 1
    return 0
    
def get_paths(dir_path):
    file_paths = []
    audio_paths = []
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            if filename.endswith(".mp3"):
                audio_paths.append(os.path.join(root, filename))
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            if not filename.endswith(".mp3"):
                filepath = os.path.join(root, filename)
                audio_path_index = contains_any_index(root, audio_paths)
                if not audio_path_index == 0:
                    file_paths.append((filepath, audio_paths[audio_path_index - 1]))
    # returning all file paths
    return file_paths

In [4]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
base_path = "/content/drive/MyDrive"
paths = get_paths(base_path)


NameError: ignored

In [ ]:
class SectionName(Enum):
    General = "[General]"
    Editor = "[Editor]"
    Metadata = "[Metadata]"
    Difficulty = "[Difficulty]"
    Events = "[Events]"
    TimingPoints = "[TimingPoints]"
    Colours = "[Colours]"
    HitObjects = "[HitObjects]"


@dataclass
class Section:

    def value(self, value):
        if value.strip().isnumeric():
            return int(value)
        else:
            try:
                return float(value)
            except ValueError:
                return value

    def parse_line(self, line: str):
        ...


@dataclass
class HitSample:
    normalSet: int = 0  # SampleSet
    additionSet: int = 0  # SampleSet
    index: int = 0
    volume: int = 0
    filename: Optional[str] = None

    # def set(self, normalSet: int, additionSet: int, index: int, volume: int, filename: Optional[str] = ""):
    #     self.normalSet = normalSet
    #     self.additionSet = additionSet
    #     self.index = index
    #     self.volume = volume
    #     self.filename = filename

    def __str__(self):
        if self.filename is not None:
            return str(f"{self.normalSet}:{self.additionSet}:{self.index}:{self.volume}:{self.filename}:")
        else:
            return str(f"{self.normalSet}:{self.additionSet}:{self.index}:{self.volume}:")


@dataclass
class General(Section):
    AudioFilename: Optional[str] = None
    AudioLeadIn: int = 0
    AudioHash: Optional[str] = None  # deprecated
    PreviewTime: int = -1
    Countdown: int = 1
    SampleSet: str = "Normal"
    StackLeniency: float = 0.7
    Mode: int = 0
    LetterboxInBreaks: int = 0
    StoryFireInFront: int = 1  # deprecated
    UseSkinSprites: int = 0
    AlwaysShowPlayfield: int = 0  # deprecated
    OverlayPosition: str = "NoChange"
    SkinPreference: Optional[str] = None
    EpilepsyWarning: int = 0
    CountdownOffset: int = 0
    SpecialStyle: int = 0
    WidescreenStoryboard: int = 0
    SamplesMatchPlaybackRate: int = 0

    def parse_line(self, line: str):
        members = line.split(':')
        self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Editor(Section):
    Bookmarks: Optional[List[int]] = None
    DistanceSpacing: float = 1.22  # between 0.1 and 2.0
    BeatDivisor: int = 4
    GridSize: int = 4
    TimelineZoom: float = 1.0  # between 0.1 and 8.0

    def parse_line(self, line: str):
        members = line.split(':')
        if members[0] == "Bookmarks":
            self.Bookmarks = [self.value(x) for x in members[1].split(",")]
        else:
            self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Metadata(Section):
    Title: Optional[str] = None
    TitleUnicode: Optional[str] = None
    Artist: Optional[str] = None
    ArtistUnicode: Optional[str] = None
    Creator: Optional[str] = None
    Version: Optional[str] = None
    Source: Optional[str] = None
    Tags: Optional[List[str]] = None
    BeatmapID: int = 0
    BeatmapSetID: int = 0

    def parse_line(self, line: str):
        members = line.split(':')
        if members[0] == "Tags":
            self.Tags = [x for x in members[1].split(" ")]
        else:
            self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Difficulty(Section):
    HPDrainRate: float = 5.0
    CircleSize: float = 5.0
    OverallDifficulty: float = 5.0
    ApproachRate: float = 5.0
    SliderMultiplier: float = 1.4
    SliderTickRate: float = 1.0

    def parse_line(self, line: str):
        members = line.split(':')
        self.__setattr__(members[0], self.value(members[1]))


class EventParams:
    pass


class Event(Section):
    eventType: str
    startTime: int
    eventParams: List[EventParams]


class Background(EventParams):
    filename: str
    xOffset: int
    yOffset: int


class Video(EventParams):
    Video: 1
    startTime: int
    filename: str
    xOffset: int
    yOffset: int


class Pause(EventParams):
    # 2:Break TODO check wiki because sintaxe is strange
    Break: 2
    startTime: int
    endTime: int


#  TODO
class Storyboard(EventParams):
    pass


@dataclass
class TimingPoint(Section):
    time: int = 0
    beatLength: float = 0
    meter: int = 0
    sampleSet: int = 1  # SampleSet = SampleSet.Normal.value
    sampleIndex: int = 0
    volume: int = 1
    uninherited: int = 0
    effects: int = 0  # Effect = None
    # bpm: Optional[int] = None

    def parse_line(self, line: str):
        members = line.split(",")
        self.time = self.value(members[0])
        self.beatLength = self.value(members[1])
        self.meter = self.value(members[2])
        self.sampleSet = self.value(members[3])
        self.sampleIndex = self.value(members[4])
        self.volume = self.value(members[5])
        self.uninherited = self.value(members[6])
        self.effects = self.value(members[7])
        # self.calculate_bpm()

    # def calculate_bpm(self):
    #     self.bpm = round(60000 / self.beatLength)

    def calculate_beat_length(self, bpm: int):
        self.beatLength = 60000 / bpm

    def __str__(self):
        return ",".join([str(value) for value in self.__dict__.values() if value != None])


class Colour:
    R: int
    G: int
    B: int


# TODO check wiki for colours
@dataclass
class ColourSection(Section):
    colours: List[Colour]
    slider_body: Colour
    slider_track_override: Colour
    slider_border: Colour

    # SliderTrackOverride
    # SliderBorder
    def parse_line(self, line):
        members = line.split(":")
        isCombo = line.startswith("Combo")
        split = members[1].split(",")
        if len(split) != 3 or len(split) != 4:
            print(" invalid color")
            return -1
        assert 0 <= split[0] <= 255
        assert 0 <= split[1] <= 255
        assert 0 <= split[2] <= 255

        if isCombo:
            # {"R": split[0], "G": split[1], "B": split[2]}
            self.colours.append(Colour(R=split[0], G=split[1], B=split[2]))
        else:
            # do nothing for the moment
            pass


@dataclass
class HitObject(Section):
    x: int = 0
    y: int = 0
    time: int = 0
    type: int = 0
    hitSound: int = 0
    hitSample: Optional[str] = HitSample().__str__()

    def __str__(self):
        return f"{self.x},{self.y},{self.time},{self.type},{self.hitSound},{self.hitSample}"

    def get_hit_sample(self, line) -> str:
        if self.has_hit_sample(line):
            return line
        return "0:0:0:0:0:"

    def has_hit_sample(self, line) -> bool:
        if type(line) == int or type(line) == float:
            return False
        else:
            return True


@dataclass
class Cercle(HitObject):

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])
        self.hitSample = self.get_hit_sample(self.value(members[-1]))


@dataclass
class Spinner(HitObject):
    endTime: int = 0

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])
        self.endTime = self.value(members[5])

        self.hitSample = self.get_hit_sample(self.value(members[-1]))


@dataclass
class CurvePoint:
    x: int = 0
    y: int = 0

    def __str__(self):
        return f"{self.x}:{self.y}"


@dataclass
class Slider(HitObject):
    curvePoints: List[CurvePoint] = None
    slides: int = 0
    length: float = 0.0
    edgeSounds: str = ""
    edgeSets: str = ""
    curveType: str = ""

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])

        # Parse slider points
        points = (members[5] or '').split('|')
        self.curveType = points[0]
        self.curvePoints = []
        if len(points):
            for i in range(1, len(points)):
                coordinates = points[i].split(':')
                curve_point = CurvePoint()
                curve_point.x = self.value(coordinates[0])
                curve_point.y = self.value(coordinates[1])
                self.curvePoints.append(curve_point)
                # self.curvePoints.append(curve_point.__str__())

        # Parse repeat slides bumber & length
        self.slides = int(members[6])
        self.length = int(round(float(members[7])))

        # Parse edgeSounds
        if len(members) > 9:
            if members[8]:
                self.edgeSounds = members[8]

            # Parse edgeSets
            if members[9]:
                self.edgeSets = members[9]

        self.hitSample = self.get_hit_sample(self.value(members[-1]))

In [ ]:
class Parser:
    def __init__(self):
        self.file_format = ""
        self.general = General()
        self.editor = Editor()
        self.metadata = Metadata()
        self.difficulty = Difficulty()
        self.events: List[Event] = []
        self.timing_points: List[TimingPoint] = []
        self.colours: List[ColourSection] = []
        self.hit_objects: List[HitObject] = []

        self.osu_section = ""

    def parse_hit_object_type(self, line):
        _type = int(line.split(",")[3].strip())
        # https://osu.ppy.sh/wiki/fr/Client/File_formats/Osu_%28file_format%29#type
        # convert in bit
        # 0: Cercle
        # 1: Slider
        # 3:Spinner
        # 7 osu mania
        if _type & 1:
            cercle = Cercle()
            cercle.parse_line(line)
            return cercle
        elif _type & 2:
            slider = Slider()
            slider.parse_line(line)
            return slider
        elif _type & 8:
            spinner = Spinner()
            spinner.parse_line(line)
            return spinner
        # elif _type & 128:
        #     print("mania")
        else:
            cercle = Cercle()
            cercle.parse_line(line)
            print("unknown type:", _type)
            return cercle

    def parse_line(self, line: str):
        line = line.strip()
        if not line:
            return

        match = re.search(r"\[(.*?)\]", line)
        if match:
            self.osu_section = match.group(0)
            return
        match = re.match('^osu file format (v[0-9]+)$', line)
        if match:
            # self.file_format = line
            self.file_format = match.group(1)
            return
        if self.osu_section == SectionName.General.value:
            self.general.parse_line(line)
        elif self.osu_section == SectionName.Editor.value:
            self.editor.parse_line(line)
        elif self.osu_section == SectionName.Metadata.value:
            self.metadata.parse_line(line)
        elif self.osu_section == SectionName.Difficulty.value:
            self.difficulty.parse_line(line)
        # elif self.osu_section == SectionName.Events.name:
        #     self.events_section.append(line)
        elif self.osu_section == SectionName.TimingPoints.value:
            timing_point = TimingPoint()
            timing_point.parse_line(line)
            self.timing_points.append(timing_point)
        # elif self.osu_section == SectionName.Colours.name:
        #     self.colours_section.append(line)
        elif self.osu_section == SectionName.HitObjects.value:
            hit_obj = self.parse_hit_object_type(line)
            self.hit_objects.append(hit_obj)

    def parse_file(self, file):
        if os.path.isfile(file):
            with codecs.open(file, 'r', encoding="utf-8") as file:
                line = file.readline()
                while line:
                    self.parse_line(line)
                    line = file.readline()

In [ ]:
def scale_beatmap(hitpoints:List[HitObject]):
    # we take 7min30s for each beatmap
    duration = 7.5*60 
    new_hitpoints = []
    for h in hitpoints:
      if h.time <= duration * 1000:
        new_hitpoints.append(h)
    # hitpoints = [x for x in hitpoints if x[2] <= duration * 1000]
    return new_hitpoints

In [ ]:
def load_beatmap_attributes(path):
    cols = ["x", "y", "time", "type", "endtime", "x2", "2", "x3", "y3", "x4", "y4", "slide", "length"]

    parser = Parser()
    parser.parse_file(path)
    max_hit_object = 4000
    data = np.zeros((13, max_hit_object), dtype=object)

    hitpoints = scale_beatmap(parser.hit_objects)

    for (i, o) in enumerate(hitpoints):

        if i < max_hit_object:

            data[0][i] = o.x
            data[1][i] = o.y
            data[2][i] = o.time
            data[3][i] = o.type

            if isinstance(o, Cercle):
                pass
            elif isinstance(o, Spinner):
                data[4][i] = o.endTime
            elif isinstance(o, Slider):
                data[5][i] = o.curvePoints[0].x
                data[6][i] = o.curvePoints[0].y

                if len(o.curvePoints) > 1:
                    data[7][i] = o.curvePoints[1].x
                    data[8][i] = o.curvePoints[1].y

                if len(o.curvePoints) > 2:
                    data[9][i] = o.curvePoints[2].x
                    data[10][i] = o.curvePoints[2].y

                data[11][i] = o.slides
                data[12][i] = o.length

    return data, parser.difficulty.OverallDifficulty

In [ ]:
def load_melspectrogram(audio_path, plot=False):
    y, sr = librosa.load(audio_path, sr=22050)
    melspectrogram = np.zeros((128, 20000), dtype=float)
    melspectrogram_full = librosa.feature.melspectrogram(y=y, sr=sr)
    melspectrogram[:, :melspectrogram_full.shape[1]] = melspectrogram_full[:, :20000]
    if plot:
        fig, ax = plt.subplots()
        S_dB = librosa.power_to_db(melspectrogram, ref=np.max)
        img = librosa.display.specshow(S_dB, x_axis='time',
                                       y_axis='mel', sr=sr, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        plt.show()
    return melspectrogram.transpose()

In [ ]:
def normalize(img):
    '''
    Normalizes an array
    (subtract mean and divide by standard deviation)
    '''
    eps = 0.001
    if np.std(img) != 0:
        img = (img - np.mean(img)) / np.std(img)
    else:
        img = (img - np.mean(img)) / eps
    return img

In [ ]:
def load_beatmaps_and_spectrograms(paths: List):
    arr = []
    diff = []
    spectrograms = []
    for i,path in enumerate(paths):
      if i >= 100:
        break
      else:
        df_temp, difficulty = load_beatmap_attributes(path[0])
        df_temp = df_temp.transpose()
        spectrogram = load_melspectrogram(path[1])
        spectrogram = normalize(spectrogram)
        arr.append(df_temp)
        diff.append(difficulty)
        spectrograms.append(spectrogram)
    diff = np.array(diff, dtype=float)
    return arr, spectrograms, diff

NameError: ignored

In [ ]:
df, spectrograms, diff = load_beatmaps_and_spectrograms(paths)

<ipython-input-30-caa49d2e152f>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=22050)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
x_train = spectrograms[:int(len(spectrograms) * 0.8)]
x_train = np.array(x_train, dtype=float)
y_train = df[:int(len(df) * 0.8)]
y_train = np.array(y_train, dtype=float)
decoder_input = np.zeros((len(y_train), 4001, 13))

In [ ]:
# Ajout des tokens de début et de fin de séquence
index = 0
start_of_sequence = np.array([-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0])
end_of_sequence = np.array([-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]).reshape((1, 1, -1))
end_of_sequence = np.repeat(end_of_sequence, 7, axis=0)

for sublist in decoder_input:
    sublist[0] = start_of_sequence
    sublist[1:] = y_train[index]

y_train = np.append(y_train, end_of_sequence, axis=1)
print(y_train[0][-1])

x_val = spectrograms[int(len(spectrograms) * 0.8):]
x_val = np.array(x_val, dtype=float)
y_val = df[int(len(df) * 0.8):]
y_val = np.array(y_val, dtype=float)
print("Taille de l'input d'entraînement : " + str(x_train.shape))
print("Taille de l'output d'entraînement : " + str(y_train.shape))
print("Taille de l'input du dataset de validation : " + str(x_val.shape))
print("Taille de l'output du dataset de validation : " + str(y_val.shape))

In [ ]:
"""

Entrées : 
Sortie : .txt
"""


def build_model(input_dim, decoder_input_shape, latent_dim):
    # Model's input
    input_spectrogram = layers.Input((None, input_dim), name="input")
    # Expand the dimension to use 2D CNN.
    x = layers.Reshape((-1, input_dim, 1), name="expand_dim")(input_spectrogram)
    # Convolution layer 1
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 41],
        strides=[2, 2],
        padding="same",
        use_bias=False,
        name="conv_1",
    )(x)
    x = layers.BatchNormalization(name="conv_1_bn")(x)
    x = layers.ReLU(name="conv_1_relu")(x)
    # Convolution layer 2
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 21],
        strides=[1, 2],
        padding="same",
        use_bias=False,
        name="conv_2",
    )(x)
    x = layers.BatchNormalization(name="conv_2_bn")(x)
    x = layers.ReLU(name="conv_2_relu")(x)
    # Reshape the resulted volume to feed the RNNs layers
    x = layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)

    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(x)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, decoder_input_shape))

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = layers.Dense(13, activation='relu')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = keras.Model([input_spectrogram, decoder_inputs], decoder_outputs)
    return model

In [ ]:
model = build_model(input_dim=128, decoder_input_shape=13, latent_dim=256)

In [ ]:
model.compile(
    optimizer="adam",
    loss=keras.losses.MeanAbsoluteError(),
)

In [ ]:
model.summary()

In [ ]:
# train model
model.fit(
    [x_train, decoder_input],
    y_train,
    batch_size=64,
    epochs=5,
)

In [ ]:
# Save model
model.save("OsuMapCreator")

In [ ]:
model.predict()